### json 불러와서 저장하기

In [1]:
import numpy as np
import pandas as pd
import json
import jpype
import glob
from random import shuffle
from bs4 import BeautifulSoup as bs

#data_path = "data"
data_path = "data(test)"

file_list = glob.glob("%s/*.json" % data_path)

json_train=[]

shuffle(file_list)
for json_file_name in file_list:
    json_file = json.loads(open(json_file_name).read())
    json_train += json_file["articles"]

### json에서 author, author 형태소, forumid 추출

In [2]:
from konlpy.tag import Mecab

mecab = Mecab()

labeled_train = []

for cnt, article in enumerate(json_train):
    if cnt % 10000 == 0:
        print cnt

    labeled_train.append({
            "istroll": article["is_troll"],
            "author": article["author"],
            "author_pos": ["%s_%s" % (word, pos) for word, pos in mecab.pos(article["author"])],
            "forumid": article["forumid"],
            "pk": article["pk"]
        })

labeled_train = pd.DataFrame.from_dict(labeled_train)
labeled_train = labeled_train.set_index('pk')

labeled_train

0


,author,author_pos,forumid,istroll
pk,,,,
1,세부이장,"[세부_NNG, 이장_NNG]",마을회관밭,False
2,관리자,[관리자_NNG],잡담밭,False
3,snoopy,[snoopy_SL],잡담밭,False
4,ISCARIOT,[ISCARIOT_SL],정치사회밭,False
5,1234,[1234_SN],연예인밭,False
6,호놀룰루,[호놀룰루_NNP],정치사회밭,False
7,영양죽,"[영_NNG, 양죽_NNG]",잡담밭,False
8,영양죽,"[영_NNG, 양죽_NNG]",연예인밭,False
9,fakebook,[fakebook_SL],잡담밭,False


### forumid와 author을 label encoder로 숫자로 바꿔줌

In [3]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

labeled_train["forumid"] = le.fit_transform(labeled_train["forumid"])
labeled_train["author"] = le.fit_transform(labeled_train["author"])

labeled_train

,author,author_pos,forumid,istroll
pk,,,,
1,49,"[세부_NNG, 이장_NNG]",3,False
2,22,[관리자_NNG],5,False
3,11,[snoopy_SL],5,False
4,2,[ISCARIOT_SL],6,False
5,0,[1234_SN],4,False
6,68,[호놀룰루_NNP],6,False
7,57,"[영_NNG, 양죽_NNG]",5,False
8,57,"[영_NNG, 양죽_NNG]",4,False
9,7,[fakebook_SL],5,False


###bag of words

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

def make_bag_of_words(labeled_train, max_features, col_name):
    vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,
                                stop_words = None, max_features=max_features)
    
    sentences = labeled_train[col_name].apply(lambda x:" ".join(x))
    train_data_features = vectorizer.fit_transform(sentences).toarray()

    col = ["bow_%s_%s" % (col_name, data) for data in vectorizer.get_feature_names()]
    #col = ["bow%s%s" % (col_name, str(i)) for  in range(0, max_features)]
    df_bow = pd.DataFrame(train_data_features, columns = col, index=labeled_train.index)
    
    labeled_train = pd.concat([labeled_train, df_bow],axis=1)
    
    return labeled_train

##LDA

In [5]:
import gensim
from gensim import corpora, models
from gensim.corpora import TextCorpus, MmCorpus, Dictionary
from nltk.corpus import stopwords
from optparse import OptionParser

def make_lda(train, keep_n, num_topics, col_name):

    data = train[col_name]

    dictionary = corpora.Dictionary(data)
    dictionary.filter_extremes(keep_n=keep_n)

    corpus = [dictionary.doc2bow(text) for text in data]

    print("Make Lda..")

#    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, chunksize=1000, passes=1,distributed = True)
    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, chunksize=1000, passes=1)
    
    num = len(train)    
    df = []
    
    for i in range(0,num):
        if i % 10000 == 0:
            print(i)
        
        temp = [i[1] for i in lda.get_document_topics(corpus[i],minimum_probability=0)]
        #temp = max(lda.get_document_topics(corpus[i]),key=lambda item:item[1])[0]
        df.append(temp)
    
    col = ["lda_%s_%d" % (col_name, data) for data in range(0, num_topics)]
    df = pd.DataFrame(df, columns = col)
    df.index = train.index
    
    train = pd.concat([train, df], axis=1)
        
    return train

##Word2Vec

In [8]:
from gensim.models import word2vec

def make_feature_vec(words, model, num_features):
    
    feature_vec = np.zeros((num_features,), dtype = "float32")
    
    nwords = 0
    
    index2word_set = set(model.index2word)
    
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            feature_vec = np.add(feature_vec, model[word])
    
    if nwords != 0:
        feature_vec = np.divide(feature_vec, nwords)
    
    return feature_vec

def get_avg_feature_vecs(texts, model, num_features):
    
    counter = 0
    
    text_feature_vecs = np.zeros((len(texts), num_features), dtype = "float32")
    
    for i, text in enumerate(texts):
        
        if i % 10000 == 0:
            print(i)
        
        text_feature_vecs[i] = make_feature_vec(text , model, num_features)
        
    return text_feature_vecs

def make_word2vec(train, col_name):
    num_features = 300
    min_word_count = 40
    num_workers = 4
    context = 10
    downsampling = 1e-3
    
    sentences = " ".join(train[col_name].apply(lambda x:" ".join(x)))
    
    model = word2vec.Word2Vec(sentences, workers = num_workers, size = num_features,\
                             min_count = min_word_count, window = context, sample = downsampling)
    
    col = ["word2vec_%s_%d" % (col_name, data) for data in range(0, num_features)]
    
    train_feature = get_avg_feature_vecs(train[col_name].apply(lambda x:" ".join(x)), model, num_features)
    train_feature = pd.DataFrame(train_feature, index = train.index, columns = col)
    
    train = pd.concat([train, train_feature], axis = 1)
    
    return train

In [9]:
max_features = 1000
# #%time labeled_train = make_lda(labeled_train, 1000, 200, "author_pos")
# #%time labeled_train = make_bag_of_words(labeled_train, 1000, "author_pos")

%time labeled_train = make_word2vec(labeled_train, "author_pos")
labeled_train

0
CPU times: user 16.2 ms, sys: 8.18 ms, total: 24.4 ms
Wall time: 18.6 ms


,author,author_pos,forumid,istroll,word2vec_author_pos_0,word2vec_author_pos_1,word2vec_author_pos_2,word2vec_author_pos_3,word2vec_author_pos_4,word2vec_author_pos_5,...,word2vec_author_pos_290,word2vec_author_pos_291,word2vec_author_pos_292,word2vec_author_pos_293,word2vec_author_pos_294,word2vec_author_pos_295,word2vec_author_pos_296,word2vec_author_pos_297,word2vec_author_pos_298,word2vec_author_pos_299
pk,,,,,,,,,,,,,,,,,,,,,
1,49,"[세부_NNG, 이장_NNG]",3,False,0.000722,0.001044,-0.000005,-0.001175,0.000497,-0.000507,...,-0.000882,-0.000420,-0.000158,-0.000477,0.000173,0.000221,0.000996,0.000835,-0.000714,-0.000409
2,22,[관리자_NNG],5,False,0.000773,0.001009,0.000031,-0.001205,0.000558,-0.000479,...,-0.000856,-0.000660,-0.000074,-0.000343,0.000388,0.000360,0.001237,0.000838,-0.000934,-0.000593
3,11,[snoopy_SL],5,False,0.001591,0.000221,-0.000962,-0.000966,-0.001580,-0.001002,...,-0.000610,0.000170,0.000316,-0.001386,0.001191,-0.000610,0.000805,0.001026,-0.001041,-0.000241
4,2,[ISCARIOT_SL],6,False,0.001591,0.000221,-0.000962,-0.000966,-0.001580,-0.001002,...,-0.000610,0.000170,0.000316,-0.001386,0.001191,-0.000610,0.000805,0.001026,-0.001041,-0.000241
5,0,[1234_SN],4,False,0.001502,0.000720,-0.000211,-0.001065,-0.000231,-0.000958,...,-0.000738,-0.000395,0.000204,-0.000470,0.000715,0.000191,0.001083,0.000928,-0.000869,-0.000321
6,68,[호놀룰루_NNP],6,False,0.001472,0.000886,0.000039,-0.001099,0.000219,-0.000944,...,-0.000781,-0.000583,0.000167,-0.000165,0.000557,0.000458,0.001175,0.000896,-0.000812,-0.000348
7,57,"[영_NNG, 양죽_NNG]",5,False,0.000722,0.001044,-0.000005,-0.001175,0.000497,-0.000507,...,-0.000882,-0.000420,-0.000158,-0.000477,0.000173,0.000221,0.000996,0.000835,-0.000714,-0.000409
8,57,"[영_NNG, 양죽_NNG]",4,False,0.000722,0.001044,-0.000005,-0.001175,0.000497,-0.000507,...,-0.000882,-0.000420,-0.000158,-0.000477,0.000173,0.000221,0.000996,0.000835,-0.000714,-0.000409
9,7,[fakebook_SL],5,False,0.001591,0.000221,-0.000962,-0.000966,-0.001580,-0.001002,...,-0.000610,0.000170,0.000316,-0.001386,0.001191,-0.000610,0.000805,0.001026,-0.001041,-0.000241


### predictor, model 생성

In [11]:
from sklearn.ensemble import RandomForestClassifier

label = 'istroll'
pre = labeled_train.columns.drop(['author_pos', 'author', label])

model = RandomForestClassifier(n_estimators=10, n_jobs=6)

###cross validation

In [12]:
from sklearn import cross_validation

cv_value = 3

scores = cross_validation.cross_val_score(model, labeled_train[pre], labeled_train[label], cv=cv_value, scoring="roc_auc")

cv_result = scores.mean()

print(cv_result)

0.858544474022


## Result

 * author를 Label Encoder로 변환: 0.67331271058
 * author의 morphs를 Bag of Words(feature = 1,000 개)로 변환: 0.733333689652
 * author의 pos를 Bag of Words(feature = 1,000 개)로 변환: 0.769833271386
 * 데이터를 shuffle하고 pos를 Bag of Words(feautre = 1,000개)로 변환: 0.81172740086
 
 
 * author의 pos를 LDA로 변환 (num_topics = 200, keep_n = 1,000) : 0.748718319834
 * 데이터를 shuffle하고 author의 pos를 LDA로 변환 (num_topics = 200, keep_n = 1,000) : 0.772313052394
 
 * 데이터 shuffle 후 word2vec(num_features = 300) : 0.869585615353

In [51]:
import pickle

pre = labeled_train.columns.drop(['author_pos', 'author'])

pickle.dump(labeled_train[pre], open("author, forum_%d.p" % max_features, "wb"), protocol = pickle.HIGHEST_PROTOCOL)